In [214]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pandas as pd
import re

df = pd.read_csv(
    "https://raw.githubusercontent.com/plotly/datasets/master/Mining-BTC-180.csv"
)

headerColor = "#1a1a1a"
headerFontColor = "white"
rowEvenColor = "#f5f5f5"
rowOddColor = "white"
borderColor = "#e0e0e0"

for i, row in enumerate(df["Date"]):
    p = re.compile(" 00:00:00")
    datetime = p.split(df["Date"][i])[0]
    df.iloc[i, 1] = datetime

# Define your scatter plots here - add/remove as needed
# Each dict represents one scatter plot with x, y data and name
scatter_configs = [
    {"x": df["Date"], "y": df["Mining-revenue-USD"], "name": "Mining Revenue"},
    {"x": df["Date"], "y": df["Hash-rate"], "name": "Hash Rate"},
    {"x": df["Date"], "y": df["Transaction-fees-BTC"], "name": "Transaction Fees"},
    {"x": df["Date"], "y": df["Cost-per-trans-USD"], "name": "Cost per Transaction"},
    {"x": df["Date"], "y": df["Market-price"], "name": "Market Price"},
    {"x": df["Date"], "y": df["Output-volume(BTC)"], "name": "Output Volume"},
]

# Calculate grid dimensions
n_cols = 4
n_scatters = len(scatter_configs)
n_scatter_rows = (n_scatters + n_cols - 1) // n_cols  # ceiling division
total_rows = 1 + n_scatter_rows  # 1 for tables + scatter rows

# Build specs: first row is 2 tables (each spanning 2 cols), rest are scatters
specs = [[{"type": "table", "colspan": 2}, None, {"type": "table", "colspan": 2}, None]]
for _ in range(n_scatter_rows):
    specs.append([{"type": "scatter"} for _ in range(n_cols)])

# Row heights: table gets more space, scatters share the rest
row_heights = [0.3] + [0.7 / n_scatter_rows] * n_scatter_rows

fig = make_subplots(
    rows=total_rows,
    cols=n_cols,
    specs=specs,
    row_heights=row_heights,
    vertical_spacing=0.08,
    horizontal_spacing=0.05,
)

# Add first table at top left
fig.add_trace(
    go.Table(
        header=dict(
            values=[
                "Date",
                "Number<br>Transactions",
                "Output<br>Volume (BTC)",
                "Market<br>Price",
                "Hash<br>Rate",
                "Cost per<br>trans-USD",
                "Mining<br>Revenue-USD",
                "Transaction<br>fees-BTC",
            ],
            font=dict(size=10),
            align="left",
            line_color=borderColor,
            line_width=1,
        ),
        cells=dict(
            values=[df[k].tolist() for k in df.columns[1:]],
            fill_color=[[rowOddColor, rowEvenColor] * len(df)],
            height=30,
            line_color=borderColor,
            line_width=1,
            align="left",
        ),
    ),
    row=1,
    col=1,
)

# Add second table at top right
fig.add_trace(
    go.Table(
        header=dict(
            values=[
                "Date",
                "Number<br>Transactions",
                "Output<br>Volume (BTC)",
                "Market<br>Price",
                "Hash<br>Rate",
                "Cost per<br>trans-USD",
                "Mining<br>Revenue-USD",
                "Transaction<br>fees-BTC",
            ],
            font=dict(size=10),
            align="left",
            line_color=borderColor,
            line_width=1,
        ),
        cells=dict(
            values=[df[k].tolist() for k in df.columns[1:]],
            fill_color=[[rowOddColor, rowEvenColor] * len(df)],
            height=30,
            align="left",
            line_color=borderColor,
            line_width=1,
        ),
    ),
    row=1,
    col=3,
)

# Add scatter plots dynamically
for i, config in enumerate(scatter_configs):
    row = 2 + i // n_cols  # starts at row 2 (after tables)
    col = 1 + i % n_cols
    fig.add_trace(
        go.Scatter(
            x=config["x"],
            y=config["y"],
            mode="lines",
            name=config["name"],
        ),
        row=row,
        col=col,
    )

fig.update_layout(
    height=300 + 250 * n_scatter_rows,  # dynamic height based on rows
    autosize=True,
    width=None,
    showlegend=False,
    template="ggplot2",
    hovermode="x unified",
    margin=dict(t=50),  # reduced top margin
)

fig.show()

In [ ]:
import os
import webbrowser
from jinja2 import Environment, FileSystemLoader
from dataclasses import dataclass, field
from typing import List, Dict, Any


@dataclass
class HTMLRenderer:
    template_dir: str = field(default_factory=lambda: os.path.join(os.getcwd(), "templates"))
    template_name: str = "template.html.j2"
    output_path: str = "/tmp/backtest_report.html"

    def render(self, figures: Dict[str, go.Figure], open_browser: bool = True):
        env = Environment(loader=FileSystemLoader(self.template_dir))
        template = env.get_template(self.template_name)

        html_parts = {}
        for i, (name, fig) in enumerate(figures.items()):
            html_parts[name] = fig.to_html(
                full_html=False,
                include_plotlyjs=(i == 0),
                config={"responsive": True},
                div_id=f"{name.replace('_html', '')}-plot",
            )

        html_report = template.render(**html_parts)

        with open(self.output_path, "w") as f:
            f.write(html_report)

        if open_browser:
            webbrowser.open(f"file://{self.output_path}")

        return self.output_path


def create_figure(scatter_configs: List[Dict], table_configs: List[Dict], n_cols: int = 4, table_height: float = 0.5, height: int = 800):
    n_scatters = len(scatter_configs)
    n_scatter_rows = max(1, (n_scatters + n_cols - 1) // n_cols)
    total_rows = 1 + n_scatter_rows

    specs = [[{"type": "table", "colspan": 2}, None, {"type": "table", "colspan": 2}, None]]
    for _ in range(n_scatter_rows):
        specs.append([{"type": "scatter"} for _ in range(n_cols)])

    row_heights = [table_height] + [(1 - table_height) / n_scatter_rows] * n_scatter_rows

    fig = make_subplots(
        rows=total_rows, cols=n_cols, specs=specs,
        row_heights=row_heights, vertical_spacing=0.05, horizontal_spacing=0.05,
    )

    for i, tbl in enumerate(table_configs[:2]):
        fig.add_trace(
            go.Table(
                header=dict(
                    values=tbl.get("headers", []),
                    font=dict(size=10), align="left",
                    line_color=tbl.get("border_color", "#e0e0e0"), line_width=1,
                ),
                cells=dict(
                    values=tbl.get("values", []),
                    fill_color=tbl.get("fill_color", [["#ffffff"]]),
                    height=tbl.get("row_height", 25), align="left",
                    line_color=tbl.get("border_color", "#e0e0e0"), line_width=1,
                ),
            ),
            row=1, col=1 if i == 0 else 3,
        )

    for i, config in enumerate(scatter_configs):
        fig.add_trace(
            go.Scatter(x=config["x"], y=config["y"], mode="lines", name=config["name"]),
            row=(2 + i // n_cols), col=(1 + i % n_cols),
        )

    fig.update_layout(
        height=height, autosize=True, width=None,
        showlegend=False, template="ggplot2", hovermode="x unified", margin=dict(t=20),
    )
    return fig


tables = [
    {
        "headers": ["Date", "Number<br>Transactions", "Output<br>Volume (BTC)",
                    "Market<br>Price", "Hash<br>Rate", "Cost per<br>trans-USD",
                    "Mining<br>Revenue-USD", "Transaction<br>fees-BTC"],
        "values": [df[k].tolist() for k in df.columns[1:]],
        "fill_color": [["white", "#f5f5f5"] * len(df)],
        "border_color": "#e0e0e0",
        "row_height": 25,
    },
    {
        "headers": ["Date", "Number<br>Transactions", "Output<br>Volume (BTC)",
                    "Market<br>Price", "Hash<br>Rate", "Cost per<br>trans-USD",
                    "Mining<br>Revenue-USD", "Transaction<br>fees-BTC"],
        "values": [df[k].tolist() for k in df.columns[1:]],
        "fill_color": [["white", "#f5f5f5"] * len(df)],
        "border_color": "#e0e0e0",
        "row_height": 25,
    },
]

figures = {
    "overview_html": create_figure([
        {"x": df["Date"], "y": df["Mining-revenue-USD"], "name": "Mining Revenue"},
        {"x": df["Date"], "y": df["Hash-rate"], "name": "Hash Rate"},
        {"x": df["Date"], "y": df["Transaction-fees-BTC"], "name": "Transaction Fees"},
        {"x": df["Date"], "y": df["Cost-per-trans-USD"], "name": "Cost per Transaction"},
    ], tables),
    "fees_html": create_figure([
        {"x": df["Date"], "y": df["Mining-revenue-USD"], "name": "Mining Revenue"},
        {"x": df["Date"], "y": df["Hash-rate"], "name": "Hash Rate"},
    ], tables),
    "pnl_html": create_figure([
        {"x": df["Date"], "y": df["Market-price"], "name": "Market Price"},
        {"x": df["Date"], "y": df["Output-volume(BTC)"], "name": "Output Volume"},
    ], tables),
    "overnight_html": create_figure([
        {"x": df["Date"], "y": df["Number-transactions"], "name": "Number of Transactions"},
    ], tables),
}

renderer = HTMLRenderer()
# renderer.render(figures)

Python(50810) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


'/tmp/backtest_report.html'

In [1]:
from dash import Dash, dcc, html, Input, Output, callback
import plotly.express as px

import pandas as pd

app = Dash()

df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')

app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                df['Indicator Name'].unique(),
                'Fertility rate, total (births per woman)',
                id='xaxis-column'
            ),
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='xaxis-type',
                inline=True
            )
        ], style={'width': '48%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                df['Indicator Name'].unique(),
                'Life expectancy at birth, total (years)',
                id='yaxis-column'
            ),
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='yaxis-type',
                inline=True
            )
        ], style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
    ]),

    dcc.Graph(id='indicator-graphic'),

    dcc.Slider(
        df['Year'].min(),
        df['Year'].max(),
        step=None,
        id='year--slider',
        value=df['Year'].max(),
        marks={str(year): str(year) for year in df['Year'].unique()},

    )
])


@callback(
    Output('indicator-graphic', 'figure'),
    Input('xaxis-column', 'value'),
    Input('yaxis-column', 'value'),
    Input('xaxis-type', 'value'),
    Input('yaxis-type', 'value'),
    Input('year--slider', 'value'))
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df[df['Year'] == year_value]

    fig = px.scatter(x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
                     y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
                     hover_name=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'])

    fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')

    fig.update_xaxes(title=xaxis_column_name,
                     type='linear' if xaxis_type == 'Linear' else 'log')

    fig.update_yaxes(title=yaxis_column_name,
                     type='linear' if yaxis_type == 'Linear' else 'log')

    return fig


if __name__ == '__main__':
    app.run(debug=True)


/Users/vanessamae/Documents/playground/python/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
